Show velocity as a ratio of reports processed vs count of specialists

In [ ]:
%load_ext sql

%config SqlMagic.feedback = False

%config SqlMagic.displaycon = False

In [ ]:
%%sql --save closed_reports --no-execute
SELECT 
  DATE(a.timestamp) as date,
  r.assigned_section as section
FROM cts_forms_report r
LEFT JOIN actstream_action a
ON r.id::TEXT = a.target_object_id
WHERE a.verb = 'Status:' and a.description = 'Updated from "open" to "closed"'


In [ ]:
%%sql --save intake_specialists
SELECT 
   COUNT(*) 
FROM 
   auth_user
WHERE is_staff = true AND is_active = true

In [ ]:
def get_closed_report_count(*, month='6', year='2023'):
    count = %sql --with closed_reports SELECT \
            COUNT(*) \
            FROM closed_reports \
            WHERE \
            EXTRACT(YEAR FROM date) = '{{ year }}' \
            AND EXTRACT(MONTH FROM date) = '{{ month }}' \

    return count.DataFrame().fillna(0)['count'][0]

In [ ]:
MONTHS = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
monthly_count = []
intake_specialists_sql = %sql --with intake_specialists SELECT count FROM intake_specialists
intake_specialists_count = intake_specialists_sql.DataFrame().fillna(0)['count'][0]
for month in MONTHS:
    count = get_closed_report_count(month=month)
    monthly_count.append(count/intake_specialists_count)

In [ ]:
import plotly.express as px
import pandas as pd
df1 = pd.DataFrame(dict(Month=MONTHS))
df2 = pd.DataFrame(dict(velocity=monthly_count))
fig = px.bar(df1, x=df1.Month, y=df2.velocity, title='Reports closed per intake specialist each month 2023', labels={'y':'Reports Closed'})
fig.update_layout(font_size=10, height=500)
fig.show()